In [14]:
import time
import os
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ

In [2]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/Kate's/"
csv_filename = 'raw_velocity_data_JJ.csv'

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_filename)
df = pd.DataFrame(df)

enable_photoZ_interaction = True
initialize_photoZ = False

skip_recs = [] # ['02-01-05_L23_interlaminar_intralaminar', '03-01-06_L4_intralaminar']

In [3]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [4]:
if not enable_photoZ_interaction:
    skip_recs = []
for subdir, dirs, files in os.walk(data_dir):
    if subdir in skip_recs:
        continue
    else:
        skip_recs.append(subdir)
    zda_file = None
    roi_files = []
    roi_files_spare = []
    
    # locate important file names
    for f in files:
        if f.endswith('.zda'):
            zda_file = f
        if f.startswith('rois_'):
            if f.endswith('laminar.dat'):
                roi_files.append(f)
            else:
                roi_files_spare.append(f)
        if zda_file is not None and len(roi_files) == 2:
            break
    if len(roi_files) < 1:
        roi_files = roi_files_spare
        if len(roi_files) > 0:
            roi_files = [roi_files[0]]
        
    if zda_file is not None:
        rec_id = zda_file.split('.')[0]
        # alias the rec_id
        if '_aka_' in subdir:
            rec_id = subdir.split('_aka_')[0][-8:].replace("-", "_")
            print("aliased:", rec_id)
            
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            print("\n\nOpening", zda_file)
            aPhz.open_zda_file(subdir + "/" + zda_file)
        
        date = subdir.split("/")[-1].split('\\')[0].replace('-', '_')
        date = date[5:] + '_' + date[:4]
        print(subdir)
        for roi_file in roi_files:
            print(roi_file)
            
            # open roi file
            if enable_photoZ_interaction:
                aPhz.select_roi_tab()
                aPhz.open_roi_file(roi_file)
            
            # Now load the data from the CSV needed to know the monosynaptic window
            lam_type = None
            lam_type_subset = df[(df['Date'] == date) 
                               & (df['Slice_Loc_Run'] == rec_id)]
            lam_type = list(lam_type_subset.Laminar.unique())

            
            if 'interlaminar' in roi_file and any(["Inter" in x for x in lam_type]):
                lam_type = ['Inter', 'Inter_L23', 'Inter_L5']
            elif 'intralaminar' in roi_file and "Intra" in lam_type:
                lam_type = ['Intra']
            elif 'interlaminar' in subdir and any(["Inter" in x for x in lam_type]):
                lam_type = ['Inter', 'Inter_L23', 'Inter_L5']
            elif 'intralaminar' in subdir and "Intra" in lam_type:
                lam_type = ['Intra']
            else:
                print("For " + " ".join(lam_type) + " for roi_file " + roi_file + " \n\tsubdir " + subdir + " rec " + rec_id)
                raise Exception("What is the laminar type? Unknown")
            mono_window_full = df[(df['Date'] == date) 
                             & (df['Laminar'].isin(lam_type))
                             & (df['Slice_Loc_Run'] == rec_id)]
            mono_window = pd.DataFrame(mono_window_full,
                                       columns = ['Date', 'Laminar',
                                                'Mono_Resp_Start_Time',
                                                'Mono_Resp_End_Time'])
            if mono_window.size < 1:
                raise Exception("(No row) Check laminar value " + " ".join(lam_type) + " for date " + date + " rec " + rec_id)
                continue
            mono_window = mono_window.drop_duplicates()
            if mono_window.shape[0] != 1:
                print(mono_window, mono_window.shape)
                raise Exception("Check laminar value " + " ".join(lam_type) + " for date " + date + " rec " + rec_id)
            
            # calculate and set measure window
            start = int(mono_window['Mono_Resp_Start_Time'] * 2)
            width = int(mono_window['Mono_Resp_End_Time'] * 2 - start)
            if enable_photoZ_interaction:
                aPhz.set_measure_window(start, width)
            else:
                print("Measure window:", start, width)
                
            # Save values to file via PhotoZ: monosynaptic latency, max amp, SNR, peak time
            latency_filename = subdir + "/" + "mono_latency_" + lam_type[0] + '.dat'
            print("Latency file dst:", latency_filename)
            if enable_photoZ_interaction:
                aPhz.select_latency_trace_value()
                aPhz.save_trace_values(latency_filename)
                
            maxamp_filename = subdir + "/" + "mono_maxamp_" + lam_type[0] + '.dat'
            print("maxamp_filename dst:", maxamp_filename)
            if enable_photoZ_interaction:
                aPhz.select_maxamp_trace_value()
                aPhz.save_trace_values(maxamp_filename)
                
            snr_filename = subdir + "/" + "mono_snr_" + lam_type[0] + '.dat'
            print("snr_filename dst:", snr_filename)
            if enable_photoZ_interaction:
                aPhz.select_SNR_trace_value()
                aPhz.save_trace_values(snr_filename)
                
            peaktime_filename = subdir + "/" + "mono_peaktime_" + lam_type[0] + '.dat'
            print("peaktime_filename dst:", peaktime_filename)
            if enable_photoZ_interaction:
                aPhz.select_peaktime_trace_value()
                aPhz.save_trace_values(peaktime_filename)
            
            intermed_files = [latency_filename, maxamp_filename, snr_filename, peaktime_filename]
            col_names = ['Latency_Mono_Raw', 'Amp_Mono', 'SNR_Mono', 'Peak_Time_Mono']
            
            #print("BEFORE WRITING")
            #print(df[(df['Date'] == date) 
            #       & (df['Laminar'].isin(lam_type))
            #       & (df['Slice_Loc_Run'] == rec_id)][col_names])
            
            # open these various files and store them to the pandas dataframe
            for i in range(len(intermed_files)):
                intermed_file = intermed_files[i]
                col_name = col_names[i]
                intermed_df = pd.read_csv(intermed_file,
                                         sep='\t',
                                         header=None,
                                         names=['Index',  'Values'])

                row_indices = df.index[(df['Date'] == date) 
                 & (df['Laminar'].isin(lam_type))
                 & (df['Slice_Loc_Run'] == rec_id)].tolist()

                if intermed_df['Values'].shape[0] == len(row_indices):
                    for j in range(len(row_indices)):
                        row_idx = row_indices[j]
                        df.loc[row_idx,[col_name]] = intermed_df['Values'][j]
                else:
                    print("Could not write values for " + " ".join(lam_type) 
                          + " for date " + date + " rec " + rec_id
                          + "col name" + col_name)
                    print(df[(df['Date'] == date) 
                     & (df['Laminar'].isin(lam_type))
                     & (df['Slice_Loc_Run'] == rec_id)])
                    print(intermed_df['Values'])
            
            # print("\n\nNum empty cells ", df[['Latency_Mono_Raw', 'Amp_Mono', 'SNR_Mono', 'Peak_Time_Mono']].count())
            #print("AFTER WRITE")
            #print(df[(df['Date'] == date) 
            #       & (df['Laminar'].isin(lam_type))
            #       & (df['Slice_Loc_Run'] == rec_id)][col_names])
            



Opening 02_01_05.zda
C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar
rois_01_to_20_interlaminar.dat
Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar/mono_latency_Inter.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar/mono_maxamp_Inter.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar/mono_snr_Inter.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar/mono_peaktime_Inter.dat
rois_01_to_25_intralaminar.dat
Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar/mono_latency_Intra.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-10-02\02-01-05_L23_interlaminar_intralaminar/mono_maxamp_Intra.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's

Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-12-29\09-01-02_L4_interlaminar/mono_latency_Inter.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-12-29\09-01-02_L4_interlaminar/mono_maxamp_Inter.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-12-29\09-01-02_L4_interlaminar/mono_snr_Inter.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2020-12-29\09-01-02_L4_interlaminar/mono_peaktime_Inter.dat


Opening 09_01_01.zda
C:/Users/jjudge3/Desktop/Data/Kate's/2021-01-11\09-01-01_L23_interlaminar
rois_01_to_15.dat
Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-01-11\09-01-01_L23_interlaminar/mono_latency_Inter.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-01-11\09-01-01_L23_interlaminar/mono_maxamp_Inter.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-01-11\09-01-01_L23_interlaminar/mono_snr_Inter.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-01-11\09-01-01_L2

maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-03-23\05-01-02_L5_interlaminar_intralaminar/mono_maxamp_Intra.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-03-23\05-01-02_L5_interlaminar_intralaminar/mono_snr_Intra.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-03-23\05-01-02_L5_interlaminar_intralaminar/mono_peaktime_Intra.dat


Opening 06_01_01.zda
C:/Users/jjudge3/Desktop/Data/Kate's/2021-03-23\06-01-01_L5_interlaminar
rois_01_to_21.dat
Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-03-23\06-01-01_L5_interlaminar/mono_latency_Inter.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-03-23\06-01-01_L5_interlaminar/mono_maxamp_Inter.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-03-23\06-01-01_L5_interlaminar/mono_snr_Inter.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-03-23\06-01-01_L5_interlaminar/mono_peaktime_Inter.dat


Opening 07_01_01.zda
C:/Users/jjudge3/D

maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-25\02-01-03_L23_interlaminar_intralaminar/mono_maxamp_Inter.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-25\02-01-03_L23_interlaminar_intralaminar/mono_snr_Inter.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-25\02-01-03_L23_interlaminar_intralaminar/mono_peaktime_Inter.dat


Opening 03_01_05.zda
C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-25\03-01-05_L23_interlaminar
rois_01_to_22.dat
Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-25\03-01-05_L23_interlaminar/mono_latency_Inter.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-25\03-01-05_L23_interlaminar/mono_maxamp_Inter.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-25\03-01-05_L23_interlaminar/mono_snr_Inter.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-25\03-01-05_L23_interlaminar/mono_peaktime_Inter.dat


Opening 04_01_03.zda
C:/Users/j

Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-26\07-01-03_L4_interlaminar_intralaminar/mono_latency_Inter.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-26\07-01-03_L4_interlaminar_intralaminar/mono_maxamp_Inter.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-26\07-01-03_L4_interlaminar_intralaminar/mono_snr_Inter.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-26\07-01-03_L4_interlaminar_intralaminar/mono_peaktime_Inter.dat
rois_01_to_26_intralaminar.dat
Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-26\07-01-03_L4_interlaminar_intralaminar/mono_latency_Intra.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-26\07-01-03_L4_interlaminar_intralaminar/mono_maxamp_Intra.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-26\07-01-03_L4_interlaminar_intralaminar/mono_snr_Intra.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-05-26\07-01-0

maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-06-29\02-01-02_L5_interlaminar_intralaminar/mono_maxamp_Inter.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-06-29\02-01-02_L5_interlaminar_intralaminar/mono_snr_Inter.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-06-29\02-01-02_L5_interlaminar_intralaminar/mono_peaktime_Inter.dat


Opening 04_01_02.zda
C:/Users/jjudge3/Desktop/Data/Kate's/2021-06-29\04-01-02_L5_interlaminar_intralaminar
rois_01_to_22_intralaminar.dat
Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-06-29\04-01-02_L5_interlaminar_intralaminar/mono_latency_Intra.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-06-29\04-01-02_L5_interlaminar_intralaminar/mono_maxamp_Intra.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-06-29\04-01-02_L5_interlaminar_intralaminar/mono_snr_Intra.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-06-29\04-01-02_L5_interlaminar_

maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-08-06\08-01-03_L23_interlaminar_intralaminar/mono_maxamp_Intra.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-08-06\08-01-03_L23_interlaminar_intralaminar/mono_snr_Intra.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-08-06\08-01-03_L23_interlaminar_intralaminar/mono_peaktime_Intra.dat
rois_01_to_23_interlaminar.dat
Latency file dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-08-06\08-01-03_L23_interlaminar_intralaminar/mono_latency_Inter.dat
maxamp_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-08-06\08-01-03_L23_interlaminar_intralaminar/mono_maxamp_Inter.dat
snr_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-08-06\08-01-03_L23_interlaminar_intralaminar/mono_snr_Inter.dat
peaktime_filename dst: C:/Users/jjudge3/Desktop/Data/Kate's/2021-08-06\08-01-03_L23_interlaminar_intralaminar/mono_peaktime_Inter.dat


Opening 05_01_06.zda
C:/Users/jjudge3/Desktop/Data/Kate's/2021-

In [7]:
# write the pandas dataframe back to csv
csv_filename = 'raw_velocity_data_JJ_edited.csv'
df.to_csv(data_dir + csv_filename, index=False)

In [36]:
# Compare KS and JJ csv files
ks_csv_filename = 'raw_velocity_data_KS.csv'
ks_df = pd.read_csv(data_dir + ks_csv_filename)
ks_df = pd.DataFrame(ks_df)
tolerance = 0.05

n_viol = [0, 0, 0, 0]
n_total = 0

for idx in ks_df.index:
    for i in range(len(col_names)):
        col = col_names[i]
        n_total += 1
        v = df[col][idx]
        ks_v = ks_df[col][idx]
        if v == 0:
            if ks_df[col][idx] == 0:
                continue
            v = 0.000000001
        if col == 'Amp_Mono':
            v /= 1000
        if col == 'Latency_Mono_Raw':
            v -= 46.5
            ks_v -= 46.5
            
        # print(col, v, ks_df[col][idx])
        if np.abs((ks_v - v) / v) > tolerance:
            print("TOLERANCE VIOLATION", col, ks_v, v)
            n_viol[i] += 1

print("% consistent to tolerance:", (n_total - sum(n_viol)) / n_total )
print(col_names)
print(n_viol)

TOLERANCE VIOLATION Latency_Mono_Raw 0.39350000000000307 0.4367999999999981
TOLERANCE VIOLATION Amp_Mono 0.00125969 0.00119951
TOLERANCE VIOLATION Latency_Mono_Raw 0.5944000000000003 0.6632000000000033
TOLERANCE VIOLATION Latency_Mono_Raw 0.7999000000000009 0.674199999999999
TOLERANCE VIOLATION Amp_Mono 0.00132701 0.00114894
TOLERANCE VIOLATION SNR_Mono 6.22358 5.39179
TOLERANCE VIOLATION Latency_Mono_Raw 0.833599999999997 0.6043000000000021
TOLERANCE VIOLATION Amp_Mono 0.00133213 0.00105528
TOLERANCE VIOLATION SNR_Mono 6.45585 5.12911
TOLERANCE VIOLATION Latency_Mono_Raw 1.0536999999999992 0.4686000000000021
TOLERANCE VIOLATION Amp_Mono 0.00103705 0.000747049
TOLERANCE VIOLATION SNR_Mono 6.25985 4.50824
TOLERANCE VIOLATION Latency_Mono_Raw 1.6246999999999971 0.6923999999999992
TOLERANCE VIOLATION Amp_Mono 0.00143731 0.000852298
TOLERANCE VIOLATION SNR_Mono 6.36324 3.77158
TOLERANCE VIOLATION Latency_Mono_Raw -0.5131999999999977 0.7406999999999968
TOLERANCE VIOLATION Amp_Mono 0.0007710

TOLERANCE VIOLATION Latency_Mono_Raw -0.6111999999999966 4.254899999999999
TOLERANCE VIOLATION Amp_Mono 0.00104523 0.0007319280000000001
TOLERANCE VIOLATION SNR_Mono 3.41165 2.38855
TOLERANCE VIOLATION Peak_Time_Mono 47.5 53.0
TOLERANCE VIOLATION Latency_Mono_Raw 0.5690000000000026 3.7597999999999985
TOLERANCE VIOLATION Latency_Mono_Raw 0.4341000000000008 4.313899999999997
TOLERANCE VIOLATION Latency_Mono_Raw 0.5242999999999967 2.9198999999999984
TOLERANCE VIOLATION Amp_Mono 0.000976975 0.0005647180000000001
TOLERANCE VIOLATION SNR_Mono 2.05179 1.18515
TOLERANCE VIOLATION Latency_Mono_Raw -1.086500000000001 3.6625999999999976
TOLERANCE VIOLATION Latency_Mono_Raw -1.4688999999999979 3.552199999999999
TOLERANCE VIOLATION Latency_Mono_Raw 9.350999999999999 2.6743000000000023
TOLERANCE VIOLATION Amp_Mono 0.000398285 0.00018656800000000002
TOLERANCE VIOLATION SNR_Mono 2.87376 1.34796
TOLERANCE VIOLATION Peak_Time_Mono 46.0 51.0
TOLERANCE VIOLATION Latency_Mono_Raw -1.3156999999999996 4.2143